# Code Quality Metric Analysis
## Imports and Helpers

In [1]:
from pygount import SourceAnalysis
import os
import json
import shutil
from dotenv import load_dotenv
from complexipy import file_complexity
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
from radon.complexity import cc_visit
import subprocess
import re
import radon
import radon.metrics

from CodeMetricCalculator import get_loc, get_cog_complexity_py, get_cog_complexity_js, get_cc_py, get_cc_js, get_halstead_py, get_halstead_js
from HelperFunction import read_config_files_index, get_function_paths, get_name

In [2]:
load_dotenv()

True

---

In [4]:
model_names = [
    "GPT-3_5-Turbo", 
    "GPT-4", 
    "DeepSeek-Coder-V2", 
    "CodeQwen1_5-7B-Chat", 
    "Artigenz-Coder-DS-6_7B"
]
type_names = ["type1", "type2", "type3"]


---
## Running on All Configs

In [16]:
configs = read_config_files_index("config_files.txt")
print(f"read {len(configs)} config files")

read 10 config files


In [1]:
measure_loc = True
measure_cyc_complexity = True
measure_cog_complexity = True
measure_halstead = True

loc_data = [] # repo_name, function_num, prompt_type, model, original loc, generated loc
cc_data = [] # repo_name, function_num, prompt_type, model, original cc, generated cc
cog_data = [] # repo_name, function_num, prompt_type, model, original cog, generated cog
halstead_data = [] # repo_name, function_num, prompt_type, model, original halstead, generated halstead

cc_functions = {'python': get_cc_py, 'JS': get_cc_js, 'TS': get_cc_js}
cogc_functions = {'python': get_cog_complexity_py, 'JS': get_cog_complexity_js, 'TS': get_cog_complexity_js}
halstead_functions = {'python': get_halstead_py, 'JS': get_halstead_js, 'TS': get_halstead_js}

for conf in tqdm(configs, desc="Configs"):
    for model_name in tqdm(model_names, desc="models", leave=False):
        for prompt_type in tqdm(type_names, desc="prompt types", leave=False):
            if prompt_type == "type3" and conf["generated_function_type3_save_dir"] == "":
                # skip type3 prompt because it doesnt exist
                continue

            paths = get_function_paths(conf, model_name, prompt_type)
            original_path, generated_path = paths['original'], paths['generated']
            repo, func_num = get_name(conf)

            if measure_loc:
                original_loc = get_loc(original_path)
                generated_loc = get_loc(generated_path)

                loc_data.append([repo, func_num, prompt_type, model_name, original_loc, generated_loc])

            if measure_cyc_complexity:
                original_cc = cc_functions[conf['language']](original_path)
                generated_cc = cc_functions[conf['language']](generated_path)

                cc_data.append([repo, func_num, prompt_type, model_name, original_cc, generated_cc])

            if measure_cog_complexity:
                original_cog = cogc_functions[conf['language']](original_path)
                generated_cog = cogc_functions[conf['language']](generated_path)

                cog_data.append([repo, func_num, prompt_type, model_name, original_cog, generated_cog])

            if measure_halstead:
                original_halstead = halstead_functions[conf['language']](original_path)
                generated_halstead = halstead_functions[conf['language']](generated_path)

                halstead_data.append([repo, func_num, prompt_type, model_name, original_halstead, generated_halstead])

In [18]:
loc_df = pd.DataFrame(loc_data, columns=["Repo", "FunctionNumber", "PromptType", "Model", "OriginalLOC", "GeneratedLOC"])
cc_df = pd.DataFrame(cc_data, columns=["Repo", "FunctionNumber", "PromptType", "Model", "OriginalCC", "GeneratedCC"])
cog_df = pd.DataFrame(cog_data, columns=["Repo", "FunctionNumber", "PromptType", "Model", "OriginalCOG", "GeneratedCOG"])
halstead_df = pd.DataFrame(halstead_data, columns=["Repo", "FunctionNumber", "PromptType", "Model", "OriginalHalstead", "GeneratedHalstead"])

In [19]:
loc_df.head()

,Repo,FunctionNumber,PromptType,Model,OriginalLOC,GeneratedLOC
0,craftship_codebox-npm,1,type1,GPT-3_5-Turbo,100,41
1,craftship_codebox-npm,1,type2,GPT-3_5-Turbo,100,50
2,craftship_codebox-npm,1,type3,GPT-3_5-Turbo,100,46
3,craftship_codebox-npm,1,type1,GPT-4,100,75
4,craftship_codebox-npm,1,type2,GPT-4,100,64


In [20]:
cc_df.head()

,Repo,FunctionNumber,PromptType,Model,OriginalCC,GeneratedCC
0,craftship_codebox-npm,1,type1,GPT-3_5-Turbo,11,12.0
1,craftship_codebox-npm,1,type2,GPT-3_5-Turbo,11,13.0
2,craftship_codebox-npm,1,type3,GPT-3_5-Turbo,11,11.0
3,craftship_codebox-npm,1,type1,GPT-4,11,13.0
4,craftship_codebox-npm,1,type2,GPT-4,11,13.0


In [21]:
cog_df.head()

,Repo,FunctionNumber,PromptType,Model,OriginalCOG,GeneratedCOG
0,craftship_codebox-npm,1,type1,GPT-3_5-Turbo,11,10
1,craftship_codebox-npm,1,type2,GPT-3_5-Turbo,11,12
2,craftship_codebox-npm,1,type3,GPT-3_5-Turbo,11,7
3,craftship_codebox-npm,1,type1,GPT-4,11,18
4,craftship_codebox-npm,1,type2,GPT-4,11,15


In [22]:
halstead_df.head()

,Repo,FunctionNumber,PromptType,Model,OriginalHalstead,GeneratedHalstead
0,craftship_codebox-npm,1,type1,GPT-3_5-Turbo,3613.812,1829.796
1,craftship_codebox-npm,1,type2,GPT-3_5-Turbo,3613.812,1924.088
2,craftship_codebox-npm,1,type3,GPT-3_5-Turbo,3613.812,1726.709
3,craftship_codebox-npm,1,type1,GPT-4,3613.812,2895.571
4,craftship_codebox-npm,1,type2,GPT-4,3613.812,2599.581


In [23]:
loc_df.to_csv("csvs/code quality metrics/LOC.csv")
cc_df.to_csv("csvs/code quality metrics/CC.csv")
cog_df.to_csv("csvs/code quality metrics/COG.csv")
halstead_df.to_csv("csvs/code quality metrics/Halstead.csv")

---